# Importeer librarys

In [36]:
# Importeer gym dingen
import gym
from gym import Env
from gym.spaces import Discrete, Box, Tuple, Dict, MultiBinary, MultiDiscrete

# Import helpers
import numpy as np
import random 
import os

# Import stable baselines dingen
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor, load_results

# Types van spaces

In [2]:
Discrete(3).sample()

0

In [3]:
Box(0,1, shape = (3,3)).sample()

array([[0.19966625, 0.44591472, 0.6499098 ],
       [0.91634494, 0.02350375, 0.8505323 ],
       [0.11668705, 0.95533556, 0.7488635 ]], dtype=float32)

In [4]:
Tuple((Discrete(3), Box(0,1, shape = (3,)))).sample()

(1, array([0.65927815, 0.67812926, 0.54394984], dtype=float32))

In [5]:
Dict({'height':Discrete(2), 'speed':Box(0,100, shape = (1,))}).sample()

OrderedDict([('height', 0), ('speed', array([83.02796], dtype=float32))])

In [6]:
MultiBinary(4).sample()

array([0, 1, 0, 1], dtype=int8)

In [7]:
MultiDiscrete([5,2,2]).sample()

array([1, 0, 1])

# Envorinment maken
- Build an agent to give us the best shower possible
- Randomly temparature
- 37 tot 39 degrees

In [8]:
class ShowerEnv(Env):
    
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low = 0, high = 100, shape = (1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        # Verander de temperatuur
        self.state += action-1
        
        # Update de resterende douchesessie duur
        self.shower_length -= 1
        
        # Bereken de reward van de actie
        if self.state >=37 and self.state <=39:
            reward = 1
        else:
            reward = -1
        
        # Controleer of de douchesessie is afgelopen
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state
    

In [9]:
env = ShowerEnv()

In [10]:
env.observation_space.sample()

array([82.17465], dtype=float32)

In [11]:
env.action_space.sample()

1

# Test enviroment

In [12]:
episodes = 20
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{}, Score:{}'.format(episode, score))
env.close()

Episode:1, Score:16
Episode:2, Score:-38
Episode:3, Score:-10
Episode:4, Score:-8
Episode:5, Score:-10
Episode:6, Score:-50
Episode:7, Score:20
Episode:8, Score:-60
Episode:9, Score:-52
Episode:10, Score:-12
Episode:11, Score:10
Episode:12, Score:-54
Episode:13, Score:4
Episode:14, Score:-46
Episode:15, Score:-36
Episode:16, Score:-4
Episode:17, Score:-60
Episode:18, Score:-34
Episode:19, Score:-22
Episode:20, Score:-60


# PPO Model trainen

In [29]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose = 1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [30]:
model.learn(total_timesteps=4000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -21.7    |
| time/              |          |
|    fps             | 707      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -25.4       |
| time/                   |             |
|    fps                  | 517         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012297406 |
|    clip_fraction        | 0.0775      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000726   |
|    learning_rate        | 0.

# A2C Model trainen

In [39]:
log_path = os.path.join('Training', 'Logs')
model = A2C('MlpPolicy', env, verbose = 1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [40]:
model.learn(total_timesteps=4000)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 60       |
|    ep_rew_mean        | -36.8    |
| time/                 |          |
|    fps                | 370      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.06    |
|    explained_variance | 0.00034  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -3.53    |
|    value_loss         | 8.4      |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 60       |
|    ep_rew_mean        | -34.6    |
| time/                 |          |
|    fps                | 372      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/                |          |
|

In [41]:
Monitor(ShowerEnv).get_episode_rewards()

[]